In [1]:
import pandas as pd
import os

In [2]:
caminho = os.path.join('..', '..', 'db_softtek') 
path_arq = os.path.abspath(caminho) 
db_analitico = '\\cubo_anl\\tbl_full_analitica.xlsx'

In [3]:
# Carrega tabela de demandas:
tbl_anl = pd.read_excel(path_arq+db_analitico, dtype='string').drop_duplicates()

In [4]:
#funções genéricas:


#Função anomes:
def calcul_anomes(df,campo_data):
    if campo_data == 'DT_ABERTURA_CHAMADO':
        nova_coluna = 'ANOMES_ABERTURA'
        df[nova_coluna] = df['DT_ABERTURA_CHAMADO'].astype(str).str[0:4] + df['DT_ABERTURA_CHAMADO'].astype(str).str[5:7]
    else:
        nova_coluna = 'ANOMES_STATUS'
        df[nova_coluna] = df['DT_STATUS_CHAMADO'].astype(str).str[0:4] + df['DT_STATUS_CHAMADO'].astype(str).str[5:7]

    return df

#Função calculo Leadtime de atendimento:
def coluna_leadtime(dias):
    if dias == '0 days':
        return 'D0'
    if dias == '1 days':
        return 'D1'
    elif dias == '2 days':
        return 'D2'
    elif dias == '3 days':
        return 'D3'
    elif dias == '4 days':
        return 'D4'
    else:
        return 'D5+'

In [5]:
#Campo de data abertura do chamado:
tbl_anl['DT_ABERTURA_CHAMADO'] = tbl_anl['DT_ABERTURA_CHAMADO'].astype('datetime64[ns]').dt.strftime("%Y-%m-%d")

#Campo de data status do Chamado:

tbl_anl['DT_STATUS_CHAMADO'] = tbl_anl['EXERCICIO']+"-"+tbl_anl['PERIODO'].str.zfill(2)+"-"+tbl_anl['DIA'].str.zfill(2)
tbl_anl['DT_STATUS_CHAMADO'] = tbl_anl['DT_STATUS_CHAMADO'].astype('datetime64[ns]').dt.strftime("%Y-%m-%d")


# Calcula anomes 
tbl_anl = calcul_anomes(tbl_anl,'DT_ABERTURA_CHAMADO')
tbl_anl = calcul_anomes(tbl_anl,'DT_STATUS_CHAMADO')
tbl_anl = tbl_anl.sort_values(['CHAMADO','DT_ABERTURA_CHAMADO','DT_STATUS_CHAMADO'])

# Criando campos de LeadTime:
tbl_anl['LEADTIME_CHAMADO'] = tbl_anl['DT_STATUS_CHAMADO'].astype('datetime64[ns]') - tbl_anl['DT_ABERTURA_CHAMADO'].astype('datetime64[ns]')
tbl_anl['LEADTIME_CALCULADO'] = tbl_anl['LEADTIME_CHAMADO'].astype(str).apply(coluna_leadtime)




In [6]:
# Indicador de volume de demandas distintas:
tbl_anl_abert = tbl_anl.drop_duplicates(subset=['CHAMADO','DIA','CONSULTOR'], keep='first')
tbl_anl_abert = tbl_anl[['ANOMES_ABERTURA','SENIORIDADE','MODULO_CustosNSULTOR','CHAMADO']].groupby(['ANOMES_ABERTURA','SENIORIDADE','MODULO_CustosNSULTOR']).agg({'CHAMADO':'count'}).reset_index()
tbl_anl_abert['INDICADOR'] = 'ABERTURAS'

tbl_anl_abert.rename(columns={'MODULO_CustosNSULTOR':'ATRIBUTO','ANOMES_ABERTURA':'ANOMES_REF',
                              'SENIORIDADE':'ATRIBUTO2'}, inplace=True)
tbl_anl_abert.head(5)

,ANOMES_REF,ATRIBUTO2,ATRIBUTO,CHAMADO,INDICADOR
0,202209,Jr,Infra,1,ABERTURAS
1,202209,Sr,Infra,2,ABERTURAS
2,202304,Bg,Compras,7,ABERTURAS
3,202304,Sr,Compras,2,ABERTURAS
4,202306,Pr,Vendas,2,ABERTURAS


In [7]:
tbl_anl = tbl_anl.drop_duplicates(subset=['CHAMADO','DIA','CONSULTOR'], keep='last')

In [8]:
tbl_anl_pendentes = tbl_anl[tbl_anl['STATUS_CHAMADO'] !='Closed']
tbl_anl_pendentes = tbl_anl_pendentes[['ANOMES_ABERTURA','SENIORIDADE','COMPLEXIDADE','CHAMADO']].groupby(['ANOMES_ABERTURA','SENIORIDADE','COMPLEXIDADE']).agg({'CHAMADO':'count'}).reset_index()
tbl_anl_pendentes['INDICADOR'] = 'PENDENTE'
tbl_anl_pendentes.rename(columns={'COMPLEXIDADE':'ATRIBUTO','ANOMES_ABERTURA':'ANOMES_REF',
                              'SENIORIDADE':'ATRIBUTO2'}, inplace=True)
tbl_anl_pendentes.head(5)

,ANOMES_REF,ATRIBUTO2,ATRIBUTO,CHAMADO,INDICADOR
0,202209,Jr,N2,1,PENDENTE
1,202209,Sr,N2,2,PENDENTE
2,202308,Ex,N3,7,PENDENTE
3,202308,Pr,N1,5,PENDENTE
4,202308,Sr,N3,10,PENDENTE


In [9]:
tbl_anl_encerr = tbl_anl[tbl_anl['STATUS_CHAMADO'] =='Closed']
tbl_anl_encerr = tbl_anl_encerr[['ANOMES_STATUS','SENIORIDADE','MODULO_CustosNSULTOR','CHAMADO']].groupby(['ANOMES_STATUS','SENIORIDADE','MODULO_CustosNSULTOR']).agg({'CHAMADO':'count'}).reset_index()
tbl_anl_encerr['INDICADOR'] = 'ENCERRAMENTO'
tbl_anl_encerr.rename(columns={'MODULO_CustosNSULTOR':'ATRIBUTO','ANOMES_STATUS':'ANOMES_REF',
                              'SENIORIDADE':'ATRIBUTO2'}, inplace=True)
tbl_anl_encerr.head(5)

,ANOMES_REF,ATRIBUTO2,ATRIBUTO,CHAMADO,INDICADOR
0,202401,Bg,Compras,15,ENCERRAMENTO
1,202401,Bg,Custos,1,ENCERRAMENTO
2,202401,Bg,Depósito,2,ENCERRAMENTO
3,202401,Bg,Dev,1,ENCERRAMENTO
4,202401,Bg,FIscal,1,ENCERRAMENTO


In [10]:
tbl_anl_long = tbl_anl[tbl_anl['STATUS_CHAMADO'] =='Closed']
tbl_anl_long = tbl_anl_long[['ANOMES_STATUS','LEADTIME_CALCULADO','MODULO_CustosNSULTOR','CHAMADO']].groupby(['ANOMES_STATUS','LEADTIME_CALCULADO','MODULO_CustosNSULTOR']).agg({'CHAMADO':'count'}).reset_index()
tbl_anl_long['INDICADOR'] = 'LEADTIME'
tbl_anl_long.rename(columns={'MODULO_CustosNSULTOR':'ATRIBUTO','ANOMES_STATUS':'ANOMES_REF',
                              'LEADTIME_CALCULADO':'ATRIBUTO2'}, inplace=True)
tbl_anl_long.head(50)

,ANOMES_REF,ATRIBUTO2,ATRIBUTO,CHAMADO,INDICADOR
0,202401,D0,Compras,6,LEADTIME
1,202401,D0,Custos,3,LEADTIME
2,202401,D0,Depósito,2,LEADTIME
3,202401,D0,Dev,1,LEADTIME
4,202401,D0,FIscal,2,LEADTIME
5,202401,D0,Financeiro,6,LEADTIME
6,202401,D0,Infra,78,LEADTIME
7,202401,D0,Manutenção,1,LEADTIME
8,202401,D0,Vendas,7,LEADTIME
9,202401,D1,Compras,2,LEADTIME


In [11]:
tbl_anl_complex = tbl_anl[tbl_anl['STATUS_CHAMADO'] =='Closed']
tbl_anl_complex = tbl_anl_complex[['ANOMES_STATUS','SENIORIDADE','COMPLEXIDADE','CHAMADO']].groupby(['ANOMES_STATUS','SENIORIDADE','COMPLEXIDADE']).agg({'CHAMADO':'count'}).reset_index()
tbl_anl_complex['INDICADOR'] = 'COMPLEXIDADE_VOL'
tbl_anl_complex.rename(columns={'COMPLEXIDADE':'ATRIBUTO','ANOMES_STATUS':'ANOMES_REF',
                              'SENIORIDADE':'ATRIBUTO2'}, inplace=True)
tbl_anl_complex.head(5)

,ANOMES_REF,ATRIBUTO2,ATRIBUTO,CHAMADO,INDICADOR
0,202401,Bg,N1,75,COMPLEXIDADE_VOL
1,202401,Bg,N2,2,COMPLEXIDADE_VOL
2,202401,Bg,N3,2,COMPLEXIDADE_VOL
3,202401,Ex,N1,15,COMPLEXIDADE_VOL
4,202401,Ex,N3,4,COMPLEXIDADE_VOL


In [12]:
tbl_anl_cargo_VOL = tbl_anl[['ANOMES_STATUS','SENIORIDADE','CHAMADO','MODULO_CustosNSULTOR']].groupby(['ANOMES_STATUS','SENIORIDADE','MODULO_CustosNSULTOR']).agg({'CHAMADO':'count'}).reset_index()
tbl_anl_cargo_VOL.rename(columns={'MODULO_CustosNSULTOR':'ATRIBUTO','ANOMES_STATUS':'ANOMES_REF',
                              'LEADTIME_CALCULADO':'ATRIBUTO2'}, inplace=True)
tbl_anl_cargo_VOL['INDICADOR'] = 'VOLUMETRIA CARGO'


tbl_anl_cargo_tma = tbl_anl[['ANOMES_STATUS','SENIORIDADE','COMPLEXIDADE','HORAS','VALOR_AT','CONSULTOR']].drop_duplicates()
tbl_anl_cargo_tma['HORAS'] = round(tbl_anl_cargo_tma['HORAS'].astype('float'),2)
tbl_anl_cargo_tma['VALOR_AT'] = round(tbl_anl_cargo_tma['VALOR_AT'].astype('float'),2)

tbl_anl_cargo_tma = tbl_anl_cargo_tma.groupby(['ANOMES_STATUS','SENIORIDADE','COMPLEXIDADE']).agg({'HORAS':'mean', 'VALOR_AT':'mean'}).reset_index()
tbl_anl_cargo_tma.head(5)

,ANOMES_STATUS,SENIORIDADE,COMPLEXIDADE,HORAS,VALOR_AT
0,202401,Bg,N1,2.9400,15.000000
1,202401,Bg,N2,0.4200,15.000000
2,202401,Bg,N3,0.4200,15.000000
3,202401,Ex,N1,4.2000,80.000909
4,202401,Ex,N3,3.2375,86.250000


In [13]:
path_arq

'c:\\Users\\andre\\OneDrive\\Área de Trabalho\\FIAP\\Global_Solution\\softtek\\db_softtek'

In [14]:
cubo_1 = pd.concat([tbl_anl_abert,tbl_anl_encerr,tbl_anl_long,tbl_anl_pendentes,tbl_anl_complex,tbl_anl_cargo_VOL], axis=0)

In [24]:
json_cb1 = cubo_1.reset_index().drop(columns='index').to_json(path_arq+'\\cubo_anl\\cubo1_consolidado.json', orient='records', indent=4)

In [25]:
tbl_anl_cargo_tma.reset_index().drop(columns='index').to_json(path_arq+'\\cubo_anl\\cubo2_consolidado.json', orient='records', indent=4)